# Лабораторная работа №3
## Выполнил студент группы Щавлев К.В. БВТ2003

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [23]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

#для пятнашек
from tabulate import tabulate
import time
from queue import PriorityQueue

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [24]:
def prefix(template):
    array = [0]*len(template)
    j = 0
    i = 1
    while i < len(template): 
        if template[j] == template[i]:
            array[i] = j+1 
            i += 1
            j += 1
        else:
            if j == 0:
                array[i] = 0
                i += 1
            else:
                j = array[j-1]
    return array
test = 'лилила'
prefix (test)


[0, 0, 1, 2, 3, 0]

In [25]:
def kmp(original,template):
    m = len(template)
    n = len(original)
    p = prefix(template)
    i = 0
    j = 0
    while i < n:
        if original[i] == template[j]:
            i += 1
            j += 1
            if j == m:
                print(f"example found, index {i-m}")
                break
        else:
            if j > 0:
                j = p[j-1]
            else:
                i += 1

    if i == n and j != m:
        print("example not found")

#### Упрощенный алгоритм Бойера-Мура

In [26]:
def go_symb(template):
    S = set()
    M = len(template)
    d = {}

    for i in range(M-2, -1, -1):
        if template[i] not in S:
            d[template[i]] = M-i-1
            S.add(t[i])

    if template[M-1] not in S:
        d[template[M-1]] = M

    d['*'] = M

    return d

In [27]:
def bm(str,template):
    N = len(str)
    M = len(template)
    d = go_symb(template)
    if N >= M:
        i = M-1

        while(i < N):
            k = 0
            j = 0
            flBreak = False
            for j in range(M-1, -1, -1):
                if str[i-k] != template[j]:
                    if j == M-1:
                        off = d[str[i]] if d.get(str[i], False) else d['*']
                    else:
                        off = d[template[j]]

                    i += off
                    flBreak = True
                    break

                k += 1
            if not flBreak:
                print(f"exmaple found, index {i-k+1}")
                break
        else:
            print("example not found")
    else:
        print("example not found")

In [28]:
a = 'here is a simple example'
t = 'example'

kmp(a,t)
bm(a,t)

example found, index 17
exmaple found, index 17


In [29]:
a = 'qwertyupopasdnjlkxzqwerewqitpowqqwertyqoitqjoi111qwertyupopasdnjlkxzqwerewqitpowqqwertyqoitqjoi'
t = '111'
all_search = dict(
    КНП = kmp,
    БМ = bm,
)
winner = ''
min_time = -1
for name,callable in all_search.items():
    ITS_TIME_TO_START = time.perf_counter_ns()
    callable(a,t)
    ITS_TIME_TO_STOP = time.perf_counter_ns()
    timeout = ITS_TIME_TO_STOP-ITS_TIME_TO_START
    if(min_time == -1 or min_time > timeout):
        min_time = timeout
        winner = name
    print(f"{name} is {timeout}")

print(f"{winner} is faster boii!")

example found, index 46
КНП is 248400
exmaple found, index 46
БМ is 47200
БМ is faster boii!


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [30]:
N = 4

# Функция, определяющая движение пятнашек
def moves(position):
    blank = position.index(0)
    # Возвращаются значения из кортежа с целой частью и остатком
    i, j = divmod(blank, N)
    # Задаем массив из смещений
    offsets = []
    # Движение вниз
    if i > 0: offsets.append(-N)     
    # Движение вверх
    if i < N - 1: offsets.append(N)  
    # Движение вправо
    if j > 0: offsets.append(-1)
    # Движение влево
    if j < N - 1: offsets.append(1)  
    for offset in offsets:
        # Заполянем пустую ячейку элементом, который мы двигаем
        swap = blank + offset
        # Проверяем на то, является ли заменяемая ячейка пустой,
        # и в случае, если у нас есть следующий элемент (не конец),
        # то выводим ее индекс, а иначе индекс последнего элемента
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))


In [31]:
# Функция, определяющая есть ли у пятнашек конечное решение
def parity(permutation):
    # Создаем набор из уникальных данных с пройденными ячейками
    # и создаем кол-во циклов прохождения по массиву ячеек
    seen, cycles = set(), 0
    for i in permutation:
        # Проверяем на наличие текущей ячейки в нашем наборе при перестановке
        # и добавляем его в набор при его отсутствии в нем
        if i not in seen:
            # Наращиваем счетчик циклов при отсутствии значений в наборе
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2

In [32]:
# Класс, определяющий параметры заданного набора
class Position:
    # Конструктор, принимающий позицию и начальную дистанцию
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    # Функция, срабатывающая при сравнении (<) объекта с другим объектом
    def __lt__(self, other):
        return self.start_distance < other.start_distance

    # Функция, срабатывающая при использовании объекта как строки
    # и создающая пятнашки в виде квадратной матрицы
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) 
for i in range(0, N*N, N))

In [33]:
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

# Проверяем, можно ли в данной расстановке найти решение. Если нет, то выводим предупреждающее сообщение
if parity(start) == 0:
    print('Задачу решить нельзя')
else:
    # Иначе заполняем наш кортеж значниями из данного набора
    start = tuple(start)

    p = Position(start, 0)

    print(p)
    print()

    # Кладем в очередь с приоритетом первоначальную позицию
    candidates = PriorityQueue()
    candidates.put(p)

    # Кортеж посещенных нами позиций
    visited = set([p])
    # Словарь из позиции в качестве ключа, значение которого в дальнейшем
    # будем заполнять теми ячейками, которые стоит переместить для получения решения
    came_from = {p.position: None}

    while p.position != SOLVED:
        # Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # Кладем в очередь все соседние позиции. Повторяем, пока не вытащим конечную позицию из очереди
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    # Последовательное решение пятнашек (путь)
    path = []
    # Сохраняем конечную позицию
    prev = p
    # Идем в обратном порядке и запоминаем очередность хода в path
    while p.position != start:
        # Запоминаем, откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

[15, 14, 10, 13, 9, 10, 14, 15]


### Вывод

Благодаря данной лабораторной работе, научился реализовывать алгоритмы поиска подстроки с помощью КМП и БМХ